# A recommender system for Steam
# Team : 
# Final Project
# CS541 - Artficial Intelligence


In [76]:
#Importin packages
import csv
import pandas as pd
import numpy as np
import matplotlib as plt
from tqdm import tqdm
import scipy.sparse as sparse
from scipy.sparse.linalg import svds

In [77]:
#Load the dataset
headers = ['user_id', 'game', 'behavior', 'play_time', '0']
steam_data = pd.read_csv('steam-200k.csv', sep=',', names=headers)

steam_data = steam_data.drop(['0'],axis=1)

steam_data = steam_data.sort_values(by=['behavior'])
# steam_data.set_index(range(0,steam_data.shape[0],1))
steam_data.head()

,user_id,game,behavior,play_time
199999,128470551,RUSH,play,1.4
70753,43955374,Orcs Must Die! 2,play,17.1
70751,43955374,XCOM Enemy Unknown,play,17.3
154701,32126281,Medieval II Total War,play,14.4
70749,43955374,Anno 2070,play,17.5


In [78]:
steam_data.reset_index(drop=True, inplace=True)
steam_data.head()

,user_id,game,behavior,play_time
0,128470551,RUSH,play,1.4
1,43955374,Orcs Must Die! 2,play,17.1
2,43955374,XCOM Enemy Unknown,play,17.3
3,32126281,Medieval II Total War,play,14.4
4,43955374,Anno 2070,play,17.5


In [95]:
steam_data_play = steam_data.loc[steam_data.behavior=='play']
steam_data_play.tail()
steam_data_purchase = steam_data.loc[steam_data.behavior=='purchase']
steam_data_purchase.head()
games_names = steam_data_purchase['game'].unique().tolist()
unique_ids = steam_data_purchase['user_id'].unique().tolist()

In [96]:
user_id_groups = steam_data_purchase.groupby("user_id")

In [81]:
print(user_id_groups.get_group(43955374))

         user_id                                               game  behavior  \
180736  43955374                                  Lego Harry Potter  purchase   
180737  43955374                                 King Arthur's Gold  purchase   
180738  43955374       Warhammer 40,000 Dawn of War  Winter Assault  purchase   
180739  43955374         Warhammer 40,000 Dawn of War  Dark Crusade  purchase   
180740  43955374                                           Overlord  purchase   
180741  43955374                                         HELLDIVERS  purchase   
180742  43955374                   Tom Clancy's Rainbow Six Vegas 2  purchase   
180743  43955374                                        Endless Sky  purchase   
180744  43955374                                        Hammerwatch  purchase   
180745  43955374                               Villagers and Heroes  purchase   
180746  43955374                           Amnesia The Dark Descent  purchase   
180747  43955374            

In [82]:
beautiful_df = pd.DataFrame(0, index=unique_ids, columns=games_names)

In [83]:
beautiful_df.shape

(12393, 5155)

In [84]:
beautiful_df['Torchlight'][10450544]

0

In [85]:
for i in tqdm(range(0, len(unique_ids),1)):
    user_id = unique_ids[i]
    user_group = user_id_groups.get_group(user_id)
    for game_name in user_group['game']:
        beautiful_df[game_name][user_id] = 1
        
            

100%|██████████| 12393/12393 [01:01<00:00, 202.13it/s]


In [129]:
beautiful_df.head(15)

,Amnesia The Dark Descent,Unturned,Aliens Colonial Marines,Champions Online,Grand Theft Auto Vice City,Quake Live,Grand Theft Auto San Andreas,Yet Another Zombie Defense,Tomb Raider Chronicles,AdVenture Capitalist,...,Realm of Perpetual Guilds,Agapan,Desktop Dungeons Goatperson DLC,Desktop Dungeons Soundtrack,Diehard Dungeon,Dr.Green,Dungeon Crawlers HD,EverQuest II Rise of Kunark,EverQuest II The Shadow Odyssey,Butsbal
10450544,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
260017289,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
168163793,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36557643,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
165608075,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
142793906,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
116564064,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
108264287,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
113300324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155919035,1,1,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [116]:
#Creating a sparse matrix of the user-item matrix
sparse_matrix = sparse.csr_matrix(beautiful_df.values, dtype='float')
print(sparse_matrix.shape)

(12393, 5155)


At a high level, SVD decomposes a matrix $R$ into the best lower rank approximation of the original matrix $R$. Mathematically, it decomposes R into a two unitary matrices and a diagonal matrix:

$$ R = U \Sigma V^T $$

where R is users's purchase matrix, $U$ is the user "features" matrix, $\Sigma$ is the diagonal matrix of singular values (essentially weights), and $V^{T}$ is the game "features" matrix. $U$ and $V^{T}$ are orthogonal, and represent different things. $U$ represents how much users "like" each feature and $V^{T}$ represents how relevant each feature is to each game.

In [146]:
#Obtaining SVD values of the user-item matrix
U, sigma, Vt = svds(sparse_matrix, k = 50)
print(U.shape, sigma.shape, Vt.shape)

(12393, 50) (50,) (50, 5155)


In [147]:
#Making sigma a diagonal matrix for multiplication
sigma = np.diag(sigma)
sigma.shape

(50, 50)

In [148]:
#Obtaining predictions for all users 
all_user_predicted_purchases = np.dot(np.dot(U, sigma), Vt) #+ user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_purchases, columns = beautiful_df.columns, index = beautiful_df.index)
preds_df.head(15)

,Amnesia The Dark Descent,Unturned,Aliens Colonial Marines,Champions Online,Grand Theft Auto Vice City,Quake Live,Grand Theft Auto San Andreas,Yet Another Zombie Defense,Tomb Raider Chronicles,AdVenture Capitalist,...,Realm of Perpetual Guilds,Agapan,Desktop Dungeons Goatperson DLC,Desktop Dungeons Soundtrack,Diehard Dungeon,Dr.Green,Dungeon Crawlers HD,EverQuest II Rise of Kunark,EverQuest II The Shadow Odyssey,Butsbal
10450544,0.245628,-0.024822,0.151417,0.040323,0.243003,0.220699,0.436825,0.071464,-0.073428,-0.004976,...,0.014082,0.014082,0.014082,0.014082,0.014082,0.014082,0.014082,0.014082,0.014082,0.014082
260017289,0.004123,0.977651,-0.004051,-0.003759,0.001610,0.007358,0.001061,-0.002767,0.002735,-0.013915,...,-0.000472,-0.000472,-0.000472,-0.000472,-0.000472,-0.000472,-0.000472,-0.000472,-0.000472,-0.000472
168163793,-0.125887,1.080981,0.006548,0.981139,-0.001461,0.161450,0.002362,-0.023201,-0.009002,-0.144628,...,0.001525,0.001525,0.001525,0.001525,0.001525,0.001525,0.001525,0.001525,0.001525,0.001525
36557643,1.256235,-0.048452,-0.066009,0.030334,1.010961,-0.199878,1.239718,-0.101756,0.138290,-0.042696,...,-0.025047,-0.025047,-0.025047,-0.025047,-0.025047,-0.025047,-0.025047,-0.025047,-0.025047,-0.025047
165608075,-0.000838,0.009785,0.005122,0.004122,-0.013515,0.052451,-0.007904,0.004036,-0.003704,0.018929,...,-0.000455,-0.000455,-0.000455,-0.000455,-0.000455,-0.000455,-0.000455,-0.000455,-0.000455,-0.000455
142793906,0.074808,-0.034375,0.030640,0.025805,-0.044708,0.077073,0.010358,0.048971,-0.068982,0.229531,...,0.005411,0.005411,0.005411,0.005411,0.005411,0.005411,0.005411,0.005411,0.005411,0.005411
116564064,0.520801,0.140030,-0.044444,-0.040438,0.028465,0.157621,-0.013159,0.021852,0.538961,-0.049687,...,-0.005911,-0.005911,-0.005911,-0.005911,-0.005911,-0.005911,-0.005911,-0.005911,-0.005911,-0.005911
108264287,0.033422,1.039845,0.045176,0.005600,-0.065959,0.012861,0.009568,0.025734,-0.007672,0.300304,...,-0.001159,-0.001159,-0.001159,-0.001159,-0.001159,-0.001159,-0.001159,-0.001159,-0.001159,-0.001159
113300324,-0.003509,0.007499,0.031279,-0.010322,0.158668,0.044834,0.264002,0.006541,-0.009786,0.027424,...,-0.005763,-0.005763,-0.005763,-0.005763,-0.005763,-0.005763,-0.005763,-0.005763,-0.005763,-0.005763
155919035,0.790184,1.026755,-0.019422,0.063810,0.827770,-0.053388,1.005504,0.253061,-0.201988,0.907041,...,0.009638,0.009638,0.009638,0.009638,0.009638,0.009638,0.009638,0.009638,0.009638,0.009638
